In [8]:
from pathlib import Path

dossier_aide = Path('./10_aides')

def get_aides_list(dossier_aide):
    isYAMLfile = lambda path : str(path).endswith('.yml') or str(path).endswith('.yaml')
    liste_fichiers = [str(aide) for aide in dossier_aide.iterdir() if isYAMLfile(aide)]
    return liste_fichiers

liste_fichiers = get_aides_list(dossier_aide)


In [9]:
liste_fichiers

['10_aides/bretagne-aide-code-permis.yml',
 '10_aides/auvergne-rhone-alpes-bourse-merite.yml',
 '10_aides/bourgogne-franche-comte-dynastage.yml',
 '10_aides/caf-ain-aide-bafa-session-approfondissement-qualification.yml',
 '10_aides/benefit_front_test.yml',
 '10_aides/caf-aide-nationale-bafa.yml',
 '10_aides/aide-au-bafa-pour-une-session-de-formation-générale-caf-de-la-haute-savoie.yml',
 '10_aides/auvergne-rhone-alpes-pass-region.yml',
 '10_aides/bretagne-jeune-international.yml',
 '10_aides/aide-au-bafa-pour-une-session-dapprofondissement-ou-de-qualification-caf-de-la-haute-savoie.yml']